# Chương 4: Phân tích Tương quan và Tương quan Chính tắc

**Mục tiêu:** Hiểu sâu sắc về mối quan hệ giữa các biến số. Không chỉ dừng lại ở tương quan cặp đôi (A và B), chúng ta sẽ bóc tách các lớp tương quan ẩn (Tương quan riêng) và mối quan hệ giữa các **nhóm biến** (CCA).

---

## 4.1. Mở rộng tư duy về Tương quan

### Case Study: Bán kem và Cháy rừng
Một nhà nghiên cứu tìm thấy dữ liệu cho thấy: Khi doanh số bán kem tăng, số vụ cháy rừng cũng tăng. Hệ số tương quan $r = 0.8$ (Rất cao).

* **Kết luận ngây thơ:** Ăn kem gây ra cháy rừng?
* **Thực tế:** Cả hai đều tăng do **Nhiệt độ** (Mùa hè). Nhiệt độ chính là biến gây nhiễu (Confounding variable).

Để loại bỏ sự hiểu lầm này, chúng ta cần công cụ mạnh hơn tương quan Pearson thông thường, đó là **Tương quan riêng (Partial Correlation)**.

### 4.1.1. Các loại tương quan cơ bản
1.  **Pearson ($r$):** Đo mối quan hệ tuyến tính giữa 2 biến định lượng. (Nhạy cảm với outlier).
2.  **Spearman ($\rho$):** Đo mối quan hệ thứ bậc (Rank). Dùng khi dữ liệu không chuẩn hoặc có outlier.

## 4.2. Tương quan Riêng (Partial Correlation)

**Định nghĩa:** Là tương quan giữa $X$ và $Y$ sau khi đã **loại bỏ** ảnh hưởng của một (hoặc nhiều) biến kiểm soát $Z$.

$$ r_{xy.z} = \frac{r_{xy} - r_{xz}r_{yz}}{\sqrt{(1-r_{xz}^2)(1-r_{yz}^2)}} $$

**Ứng dụng trong kinh doanh:**
Bạn muốn biết tác động thực sự của **Quảng cáo** ($X$) lên **Doanh số** ($Y$). Tuy nhiên, cả hai đều chịu ảnh hưởng của **Mùa vụ** ($Z$). Bạn phải tính Partial Correlation giữa Quảng cáo và Doanh số, kiểm soát biến Mùa vụ.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# --- VÍ DỤ: TƯƠNG QUAN RIÊNG ---
np.random.seed(42)
n = 200

# 1. Tạo biến 'Nhiệt độ' (Z) - Biến nguyên nhân gốc rễ
temp = np.random.normal(30, 5, n)

# 2. Tạo 'Bán Kem' (X) phụ thuộc vào Nhiệt độ
ice_cream = 2 * temp + np.random.normal(0, 10, n)

# 3. Tạo 'Số vụ cháy' (Y) cũng phụ thuộc vào Nhiệt độ
forest_fire = 0.5 * temp + np.random.normal(0, 5, n)

df_corr = pd.DataFrame({'Temp': temp, 'IceCream': ice_cream, 'Fire': forest_fire})

# --- PHÂN TÍCH ---
# 1. Tương quan Pearson thông thường
corr_matrix = df_corr.corr()
print("Ma trận tương quan Pearson (Chưa kiểm soát Nhiệt độ):")
print(corr_matrix[['IceCream', 'Fire']])

# Nhận xét: Bạn sẽ thấy IceCream và Fire có tương quan dương khá cao (dù vô lý).

Ma trận tương quan Pearson (Chưa kiểm soát Nhiệt độ):
          IceCream      Fire
Temp      0.721877  0.319476
IceCream  1.000000  0.217422
Fire      0.217422  1.000000


In [2]:
# 2. Tương quan riêng (Partial Correlation)
# Cách đơn giản: Dùng thư viện pingouin (chuyên cho thống kê) hoặc tính thủ công bằng hồi quy
# Ở đây ta dùng tư duy hồi quy để loại bỏ ảnh hưởng:
# B1: Hồi quy IceCream theo Temp -> Lấy phần dư (Residuals - phần IceCream không do Temp giải thích)
# B2: Hồi quy Fire theo Temp -> Lấy phần dư
# B3: Tính tương quan giữa 2 phần dư này

import statsmodels.api as sm

def get_residuals(y, x):
    X_const = sm.add_constant(x)
    model = sm.OLS(y, X_const).fit()
    return model.resid

res_ice = get_residuals(df_corr['IceCream'], df_corr['Temp'])
res_fire = get_residuals(df_corr['Fire'], df_corr['Temp'])

partial_r = np.corrcoef(res_ice, res_fire)[0, 1]

print(f"\nTương quan riêng (Partial Correlation) giữa Kem và Cháy rừng sau khi loại bỏ Nhiệt độ: {partial_r:.4f}")
# Kết quả sẽ xấp xỉ 0 -> Chứng minh không có mối quan hệ thực sự.


Tương quan riêng (Partial Correlation) giữa Kem và Cháy rừng sau khi loại bỏ Nhiệt độ: -0.0201


## 4.3. Phân tích Tương quan Chính tắc (Canonical Correlation Analysis - CCA)

### Khái niệm
Nếu Hồi quy tuyến tính nghiên cứu mối quan hệ: **Nhiều X -> Một Y**.
Thì CCA nghiên cứu mối quan hệ: **Nhiều X -> Nhiều Y**.

### Case Study: Sức khỏe tổ chức
Bạn muốn tìm mối liên hệ giữa:
* **Tập biến X (Sự hài lòng của nhân viên):** Lương, Phúc lợi, Cơ hội thăng tiến.
* **Tập biến Y (Hiệu quả công ty):** Doanh thu, Lợi nhuận, Giá cổ phiếu.

CCA sẽ tìm ra một tổ hợp tuyến tính của X ($U$) và một tổ hợp tuyến tính của Y ($V$) sao cho tương quan giữa $U$ và $V$ là lớn nhất.

$$ U = a_1X_1 + a_2X_2 + ... $$
$$ V = b_1Y_1 + b_2Y_2 + ... $$
$$ Maximize \: Corr(U, V) $$

In [3]:
from sklearn.cross_decomposition import CCA

# --- GIẢ LẬP DỮ LIỆU CCA ---
n_cca = 500

# Tập biến X: Đặc điểm nhân viên (Kinh nghiệm, Kỹ năng, Thái độ)
X_exp = np.random.normal(5, 2, n_cca)
X_skill = np.random.normal(70, 10, n_cca)
X_att = np.random.normal(8, 1, n_cca)

# Tập biến Y: Hiệu suất (Doanh số, KPI, Đánh giá KH)
# Giả sử có mối quan hệ ẩn: Kỹ năng cao + Thái độ tốt -> Doanh số cao + KPI cao
Y_sales = 0.5*X_skill + 2*X_att + np.random.normal(0, 5, n_cca)
Y_kpi = 0.3*X_skill + 1.5*X_att + np.random.normal(0, 2, n_cca)

# Gom vào DataFrame
X_set = pd.DataFrame({'Exp': X_exp, 'Skill': X_skill, 'Attitude': X_att})
Y_set = pd.DataFrame({'Sales': Y_sales, 'KPI': Y_kpi})

# --- THỰC HIỆN CCA ---
# Khởi tạo CCA với n_components=1 (Tìm cặp biến chính tắc đầu tiên quan trọng nhất)
cca = CCA(n_components=1)
cca.fit(X_set, Y_set)

# Biến đổi dữ liệu sang không gian chính tắc (Tính điểm số U và V)
X_c, Y_c = cca.transform(X_set, Y_set)

# Tính hệ số tương quan chính tắc
canonical_corr = np.corrcoef(X_c[:, 0], Y_c[:, 0])[0, 1]

print(f"Hệ số tương quan chính tắc (Canonical Correlation): {canonical_corr:.4f}")

# Xem trọng số (Weights/Loadings) để hiểu cấu trúc
print("\nTrọng số của tập biến X (Đóng góp vào biến tiềm ẩn U):")
print(pd.DataFrame(cca.x_weights_, index=X_set.columns, columns=['Weight_U']))

print("\nTrọng số của tập biến Y (Đóng góp vào biến tiềm ẩn V):")
print(pd.DataFrame(cca.y_weights_, index=Y_set.columns, columns=['Weight_V']))

Hệ số tương quan chính tắc (Canonical Correlation): 0.8996

Trọng số của tập biến X (Đóng góp vào biến tiềm ẩn U):
          Weight_U
Exp       0.022493
Skill     0.916505
Attitude  0.399391

Trọng số của tập biến Y (Đóng góp vào biến tiềm ẩn V):
       Weight_V
Sales  0.438785
KPI    0.898592


### Diễn giải kết quả CCA
1.  **Hệ số tương quan chính tắc:** Nếu > 0.7 là rất mạnh. Nghĩa là tồn tại một mối liên hệ chặt chẽ giữa *tổng thể năng lực nhân viên* và *tổng thể hiệu suất*.
2.  **Trọng số (Weights):**
    * Ví dụ: Nếu `Attitude` có trọng số cao nhất trong tập X và `Sales` có trọng số cao nhất trong tập Y -> "Thái độ" là yếu tố then chốt thúc đẩy "Doanh số".

---

## 4.4. Tổng kết Chương 4

1.  **Đừng tin ngay vào tương quan đơn biến:** Luôn đặt câu hỏi "Có biến thứ 3 nào chi phối không?". Hãy dùng Partial Correlation để kiểm chứng.
2.  **Tư duy tập hợp:** Khi bài toán phức tạp (Nhiều Input, Nhiều Output), hãy nghĩ đến CCA thay vì chạy hàng chục mô hình hồi quy rời rạc.

### Bài tập thực hành
Tìm bộ dữ liệu về **Sức khỏe & Dinh dưỡng** (Ví dụ: Tập X gồm lượng Protein, Carb, Fat tiêu thụ; Tập Y gồm Cân nặng, Vòng eo, Huyết áp).
* Thực hiện CCA để tìm ra chế độ dinh dưỡng nào liên quan mạnh nhất đến vóc dáng cơ thể.